<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Goxhufi, Driton

In [1]:
import nltk

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/ch07.html
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

# Information Extraction I

Unter [Information Extraction (IE)](https://en.wikipedia.org/wiki/Information_extraction) versteht man die semantische Strukturierung unstruktierter Daten, d.h. die Erkennung von (zuvor festgelegten) **Entitäten** und **Relationen**, die zwischen diesen Entitäten bestehen.

In unserem Fall haben wir es mit natürlichsprachlichen Texten zu tun, die auf diese Weise strukturiert werden sollen. Ein Extraktionsziel könnten beispielsweise Organisationen und deren Standort sein, wie im folgenden Satz aus dem NLTK-Buch:  

> **BBDO South** in **Atlanta**, which handles corporate advertising for **Georgia-Pacific**, will assume additional duties for brands like Angel Soft toilet tissue and Sparkle paper towels, said Ken Haldin, a spokesman for **Georgia-Pacific** in **Atlanta**.

Die extrahierte Information entspräche dann der folgenden Tabelle:

| **Organization** | **Location** |
| :--------------- | :----------- |
| BBDO South       | Atlanta      |
| Georgia-Pacific  | Atlanta      |

Es lassen sich also zwei NLP-Aufgaben unterscheiden, mit denen wir uns in diesem und dem nächsten Notebook beschäftigen werden:

1. **Entitätenextraktion**, d.h. die Identifikation und Kategorisierung von Entitäten (Entity Extraction)
2. **Relationsextraktion**, d.h. die Identifikation und Kategorisierung von Relationen zwischen Entitäten (Relation Extraction)

## Pipeline

Im NLTK-Buch stehen die Entitätenextraktion (Entitiy Extraction) und die Relationsextraktion (Relation Extraction) am Ende einer Verarbeitungspipeline, deren Elemente wir bis auf "relation detection" in den vorhergehenden Sitzungen schon kennengelernt haben:

<img src="https://www.nltk.org/images/ie-architecture.png" alt="Drawing" style="width: 500px"/>

Erinnern wir uns mit einem Code-Beispiel:

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag

text = "BBDO South in Atlanta, which handles corporate advertising for Georgia-Pacific, will assume additional duties for brands like Angel Soft toilet tissue and Sparkle paper towels, said Ken Haldin, a spokesman for Georgia-Pacific in Atlanta."

# 1) sentence segmentation
text = sent_tokenize(text,language='english')
# 2) tokenization
text = [word_tokenize(sent,language='english') for sent in text]
# 3) part-of-speech tagging
text = [pos_tag(sent,tagset=None, lang='eng') for sent in text]

text

[[('BBDO', 'NNP'),
  ('South', 'NNP'),
  ('in', 'IN'),
  ('Atlanta', 'NNP'),
  (',', ','),
  ('which', 'WDT'),
  ('handles', 'VBZ'),
  ('corporate', 'JJ'),
  ('advertising', 'NN'),
  ('for', 'IN'),
  ('Georgia-Pacific', 'NNP'),
  (',', ','),
  ('will', 'MD'),
  ('assume', 'VB'),
  ('additional', 'JJ'),
  ('duties', 'NNS'),
  ('for', 'IN'),
  ('brands', 'NNS'),
  ('like', 'IN'),
  ('Angel', 'NNP'),
  ('Soft', 'NNP'),
  ('toilet', 'NN'),
  ('tissue', 'NN'),
  ('and', 'CC'),
  ('Sparkle', 'NNP'),
  ('paper', 'NN'),
  ('towels', 'NNS'),
  (',', ','),
  ('said', 'VBD'),
  ('Ken', 'NNP'),
  ('Haldin', 'NNP'),
  (',', ','),
  ('a', 'DT'),
  ('spokesman', 'NN'),
  ('for', 'IN'),
  ('Georgia-Pacific', 'NNP'),
  ('in', 'IN'),
  ('Atlanta', 'NNP'),
  ('.', '.')]]

NLTK enthält keinen fertigen Chunker für das Englische. Wir behelfen uns deshalb mit dem klassifiziererbasierten Chunker aus der letzten Sitzung:

In [3]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        #self.classifier = nltk.MaxentClassifier.train( 
        #    train_set, algorithm='megam', trace=0)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

def npchunk_features(sentence, i, history):
     word, pos = sentence[i]
     if i == 0:
        prevword, prevpos = "<START>", "<START>"
     else:
        prevword, prevpos = sentence[i-1]
     if i == len(sentence)-1:
         nextword, nextpos = "<END>", "<END>"
     else:
         nextword, nextpos = sentence[i+1]
     return {"pos": pos,
             "word": word,
             "prevpos": prevpos,
             "nextpos": nextpos, 
             "prevpos+pos": "%s+%s" % (prevpos, pos),   
             "pos+nextpos": "%s+%s" % (pos, nextpos),
             "tags-since-dt": tags_since_dt(sentence, i)}  

def tags_since_dt(sentence, i):
     tags = set()
     for word, pos in sentence[:i]:
         if pos == 'DT':
             tags = set()
         else:
             tags.add(pos)
     return '+'.join(sorted(tags))

from nltk.corpus import conll2000
conlltrain = conll2000.chunked_sents('train.txt')
chunker = ConsecutiveNPChunker(conlltrain)

Hier zur Erinnerung die Performanz dieses Chunkers anhand der CoNLL2000-Testdaten:

In [4]:
conlltest = conll2000.chunked_sents('test.txt')
print(chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  92.8%%
    Precision:     86.8%%
    Recall:        91.9%%
    F-Measure:     89.3%%


Jetzt können wir `chunker.parse()` verwenden, wobei auf die Formatkonvertierung mittels `tree2conlltags` geachtet werden muss:

In [5]:
# 4) entity detection (chunking)
text = [nltk.chunk.tree2conlltags(chunker.parse(sent)) for sent in text]

text

[[('BBDO', 'NNP', 'B-NP'),
  ('South', 'NNP', 'I-NP'),
  ('in', 'IN', 'B-PP'),
  ('Atlanta', 'NNP', 'B-NP'),
  (',', ',', 'O'),
  ('which', 'WDT', 'B-NP'),
  ('handles', 'VBZ', 'B-VP'),
  ('corporate', 'JJ', 'B-NP'),
  ('advertising', 'NN', 'I-NP'),
  ('for', 'IN', 'B-PP'),
  ('Georgia-Pacific', 'NNP', 'B-NP'),
  (',', ',', 'O'),
  ('will', 'MD', 'B-VP'),
  ('assume', 'VB', 'I-VP'),
  ('additional', 'JJ', 'B-NP'),
  ('duties', 'NNS', 'I-NP'),
  ('for', 'IN', 'B-PP'),
  ('brands', 'NNS', 'B-NP'),
  ('like', 'IN', 'B-PP'),
  ('Angel', 'NNP', 'B-NP'),
  ('Soft', 'NNP', 'I-NP'),
  ('toilet', 'NN', 'I-NP'),
  ('tissue', 'NN', 'I-NP'),
  ('and', 'CC', 'O'),
  ('Sparkle', 'NNP', 'B-NP'),
  ('paper', 'NN', 'I-NP'),
  ('towels', 'NNS', 'I-NP'),
  (',', ',', 'O'),
  ('said', 'VBD', 'B-VP'),
  ('Ken', 'NNP', 'B-NP'),
  ('Haldin', 'NNP', 'I-NP'),
  (',', ',', 'O'),
  ('a', 'DT', 'B-NP'),
  ('spokesman', 'NN', 'I-NP'),
  ('for', 'IN', 'B-PP'),
  ('Georgia-Pacific', 'NNP', 'B-NP'),
  ('in', 'IN', 'B

Der letzte Verarbeitungsschritt "entity detection" beinhaltet tatsächlich nur eine **Teilaufgabe der Entitätenextraktion**; es fehlt die Kategorisierung der identifizierten Entitäten. Außerdem möchte man sich bei der Informationsextraktion gerne auf Eigennamen wie *Georgia-Pacific* beschränken und diese grob ontologisch Kategorisierung. Diese Einengung bei der Entitätenextraktion nennt man Named-Entity Recognition (NER).  

## Named-Entity Recognition (NER)

Named Entities (NE) sind **durch einen Eigennamen bezeichnete einzelne Entitäten**. *President Obama* ist die Bezeichnung für ein bestimmte, einzelne Person, *Tübingen* ist die Bezeichnung für eine bestimmte, einzelne Stadt etc. Solche Bezeichnungen in einem Text gelten als NEs. 

Dagegen sind *der erste afroamerikanische Präsident* oder *die schönste Stadt am Neckar* keine NEs, obwohl offensichtlich eine einzelne Entität gemeint sein kann. *Präsident* und *Stadt* sind sogenannte Gattungsnamen, aber keine [Eigennamen](https://de.wikipedia.org/wiki/Eigenname). Die Grenzen zwischen Eigennamen und Gattungsnamen sind aber nicht immer eindeutig. *Neckar* ist zwar ein Eigenname, aber i) *Stadt am Neckar* ist wohl eher ein Gattungsname und ii) *Neckar* außerdem nicht Teil des Chunks (weil Chunks nicht-rekursiv sind).   

Im NLTK-Buch werden die folgenden NE-Typen unterschieden.


| **NE Type**  | **Examples**                            |
| :----------- | :-------------------------------------- |
| ORGANIZATION | Georgia-Pacific Corp., WHO              |
| PERSON       | Eddy Bonte, President Obama             |
| LOCATION     | Murray River, Mount Everest             |
| DATE         | June, 2008-06-29                        |
| TIME         | two fifty a m, 1:30 p.m.                |
| MONEY        | 175 million Canadian Dollars, GBP 10.40 |
| PERCENT      | twenty pct, 18.75 %                     |
| FACILITY     | Washington Monument, Stonehenge         |
| GPE          | South East Asia, Midlothian             |

Meist beschränkt man sich aber auf eine bestimmte Teilmenge wie ORGANIZATION, PERSON und LOCATION (siehe unten).

Die Extraktion von NEs, die sogenannte **Named-Entity Recognition (NER)**, beinhaltet dann die folgenden beiden Aufgaben:

1. den **Umfang** einer NE erkennen
2. den **Typ** einer NE erkennen

Die erste Teilaufgabe wird im Prinzip bereits vom Chunker erledigt. Doch wie können wir die zweite Aufgabe angehen? Wie schwer ist diese Aufgabe überhaupt?

### Herausforderungen der Tokenerkennung

Nach dem Chunking liegt eine Liste von NPs vor und die Herausforderung besteht zunächst darin, zu entscheiden, ob es sich dabei jeweils um eine NE handelt oder nicht. 

Im Englischen kann das bis zu einem gewissen Grad an der **Schreibung des Chunks** ablesen: Wird der Kopf (d.h. in der Regel das letzte Wort im Chunk) groß geschrieben, dann liegt mit einiger Wahrscheinlichkeit eine NE vor.

Sehen wir uns als Beispiel die Chunks aus der Pipline an:

> BBDO South, Atlanta, corporate advertising, Georgia-Pacific, additional duties, brands, Angel Soft toilet tissue, Sparkle paper towels, Ken Haldin, a spokesman

*Angel Soft* ist vermutlich ein Eigenname, weil *Angel* und *Soft* im Satz groß geschrieben werden. Dagegen sieht *Angel Soft toilet tissue* nicht wie ein Eigenname aus, da *tissue* klein geschrieben ist. Das scheint zunächst auch semantisch sinnvoll zu sein, da es sich um einen Gattungssame mit einem integrierten Eigennamen handeln könnte.

Im Beispielsatz heißt es aber: 

> [...] brands like Angel Soft toilet tissue and Sparkle paper towels [...]

D.h. *Angel Soft toilet tissue* ist eigentlich der Eigenname einer Marke. Die Schreibung der Chunks ist also nicht immer ein zuverlässiger Indikator. 

Ein anderes Verfahren der Tokenerkennung besteht darin, eine **Liste bekannter NEs** zu konsultieren. Die Erstellung und Pflege dieser Liste ist aber naturgemäß aufwendig und die Abdeckung trotzdem lückenhaft, da NE stark von der Domäne abhängen und einem ständigen Wandel unterliegen.  

Aus Mangel an Alternativeb spielen diese NE-Listen aber bei der nachfolgenden Typerkennung eine wichtige Rolle.

### Herausforderungen der Typerkennung

Sind die NE-Token ermittelt, müssen sie einem Typ zugeordnet werden, also ORGANIZATION, PERSON, LOCATION oder MISC. 

Die Zuordnung kann per **Lookup in nach Typen getrennten NE-Listen** erfolgen, wobei sich wieder die Frage stellt, wie man diese NE-Listen erstellt und pflegt. 

Aber auch bei ausreichendem lexikalischem Wissen ist eine Typenzuordnung nicht immer eindeutig und die NP-Chunks diesbezüglich **ambig**. Im NLTK-Buch wird das Beispiel *North* erwähnt, das als LOCATION oder als PERSON klassifiziert werden könne. Ähnlich verhält es sich mit dem Eigennamen *Christian Dior* (PERSON oder ORGANIZATON). Welcher NE-Typ jeweils intendiert ist, muss über den **Kontext** erschlossen werden.

Ein eher übertriebenes Beispiel für die Ambiguität bei der NER liefert das NLTK-Buch:

<img src="https://www.nltk.org/images/locations.png" alt="Drawing" style="width: 500px"/>

Hier wird die Orthographie, die POS-Tags und die Chunk-Struktur eben gerade nicht berücksichtigt.

Eine **Eigenheit komplexer NEs** ist außerdem, dass sie aus NEs von einem anderen Typ bestehen können. Das NLTK-Buch nennt hier die Beispiele *Stanford University* (LOCATION/ORGANIZATION) und *Cecil H. Green Library* (PERSON/ORGANIZATION). Wenn diese komplexen Mehrwort-NEs unbekannt sein sollten, muss bei der Typenzuordnung die Struktur berücksichtigt werden, d.h. bei *Standford University* ist der Kopf *University* aber nicht der Modifizierer *Stanford* maßgeblich.   


### CoNLL-2003 Shared Task: Vorbereitung der Trainings- und Testdaten

Im Rahmen der CoNLL-Konferenz wurden 2002 und 2003 Shared Tasks für die NER veranstaltet. Im NLTK sind die Daten aus dem Jahr 2002 enthalten (`from nltk.corpus import conll2002`). Leider waren die Zielsprachen dort Spanisch und Niederländisch. Wir werden daher im Folgenden die Daten aus dem Shared Task von 2003 nutzen.

Hier vorab die Berichte dieser beiden Shared Tasks:

- 2002: https://www.aclweb.org/anthology/W02-2024/
- 2003: https://www.aclweb.org/anthology/W03-0419/

Die Daten des CoNLL-2003 Shared Task müssen nachinstalliert werden. Dafür benötigt man drei Dateien:

- `conll2003/eng.testa` (Development)
- `conll2003/eng.testb`
- `conll2003/eng.train`

Die Daten stammen übrigens aus dem [Reuters Corpus](https://trec.nist.gov/data/reuters/reuters.html), das auch für die Textklassifizierung benutzt wird und nicht frei verfügbar ist.

Glücklicherweise gibt es im NLTK einen "Reader" für Corpora im CoNLL-Format. Die Methode `ConllCorpusReader` verlangt nur die Angabe des Verzeichnisses und der Spaltentypen, schon kann man mit `iob_words()`, `iob_sentences()` etc. das Corpus wie gewohnt auslesen:

In [6]:
from nltk.corpus import ConllCorpusReader
conll2003 = ConllCorpusReader('conll2003/', '.*',columntypes = ('words', 'pos', 'chunk','ne'))

print(conll2003.iob_sents()[3])

[('West', 'NNP', 'I-NP'), ('Indian', 'NNP', 'I-NP'), ('all-rounder', 'NN', 'I-NP'), ('Phil', 'NNP', 'I-NP'), ('Simmons', 'NNP', 'I-NP'), ('took', 'VBD', 'I-VP'), ('four', 'CD', 'I-NP'), ('for', 'IN', 'I-PP'), ('38', 'CD', 'I-NP'), ('on', 'IN', 'I-PP'), ('Friday', 'NNP', 'I-NP'), ('as', 'IN', 'I-PP'), ('Leicestershire', 'NNP', 'I-NP'), ('beat', 'VBD', 'I-VP'), ('Somerset', 'NNP', 'I-NP'), ('by', 'IN', 'I-PP'), ('an', 'DT', 'I-NP'), ('innings', 'NN', 'I-NP'), ('and', 'CC', 'O'), ('39', 'CD', 'I-NP'), ('runs', 'NNS', 'I-NP'), ('in', 'IN', 'I-PP'), ('two', 'CD', 'I-NP'), ('days', 'NNS', 'I-NP'), ('to', 'TO', 'I-VP'), ('take', 'VB', 'I-VP'), ('over', 'IN', 'I-PP'), ('at', 'IN', 'B-PP'), ('the', 'DT', 'I-NP'), ('head', 'NN', 'I-NP'), ('of', 'IN', 'I-PP'), ('the', 'DT', 'I-NP'), ('county', 'NN', 'I-NP'), ('championship', 'NN', 'I-NP'), ('.', '.', 'O')]


Dummerweise wird der `ne`-Annotationslayer einfach ignoriert. `ConllCorpusReader` ist nicht dafür gedacht, vier oder mehr Spalten vorzufinden :-( 

Als Workaround kann man die Klasse `ConllCorpusReader` in der folgenden Weise erweitern:

In [7]:
from nltk.util import LazyConcatenation,LazyMap

class extendedConllCorpusReader(ConllCorpusReader):
    """
    `ConllCorpusReader` is a corpus reader for CoNLL-style files.
    This extension allows one to access any column via the extra 
    parameter `column`.
    """

    def iob_words(self, fileids=None, tagset=None, column='chunk'):
        """
        :return: a list of word/tag/IOB tuples
        :rtype: list(tuple)
        :param fileids: the list of fileids that make up this corpus
        :type fileids: None or str or list
        """
        self._require(self.WORDS, self.POS, self.CHUNK)
        def get_iob_words(grid):
            return self._get_iob_words(grid, tagset, column)
        return LazyConcatenation(LazyMap(get_iob_words, self._grids(fileids)))
    
    def iob_sents(self, fileids=None, tagset=None, column='chunk'):
        """
        :return: a list of lists of word/tag/IOB tuples
        :rtype: list(list)
        :param fileids: the list of fileids that make up this corpus
        :type fileids: None or str or list
        """
        self._require(self.WORDS, self.POS, self.CHUNK)
        def get_iob_words(grid):
            return self._get_iob_words(grid, tagset, column)
        return LazyMap(get_iob_words, self._grids(fileids))

    def _get_iob_words(self, grid, tagset=None, column='chunk'):
        pos_tags = self._get_column(grid, self._colmap['pos'])
        if tagset and tagset != self._tagset:
            pos_tags = [map_tag(self._tagset, tagset, t) for t in pos_tags]
        return list(zip(self._get_column(grid, self._colmap['words']), pos_tags,
                   self._get_column(grid, self._colmap[column])))
    
    
conll2003 = extendedConllCorpusReader('conll2003/', '.*',columntypes = ('words', 'pos', 'chunk','ne'))

print(conll2003.iob_sents('eng.testa',column='ne'))

[[], [('CRICKET', 'NNP', 'O'), ('-', ':', 'O'), ('LEICESTERSHIRE', 'NNP', 'I-ORG'), ('TAKE', 'NNP', 'O'), ('OVER', 'IN', 'O'), ('AT', 'NNP', 'O'), ('TOP', 'NNP', 'O'), ('AFTER', 'NNP', 'O'), ('INNINGS', 'NNP', 'O'), ('VICTORY', 'NN', 'O'), ('.', '.', 'O')], ...]


Nun sieht man also den `ne`-Layer an Stelle des Chunk-Layers mit dem schon bekannten IOB-Schema. Man beachte aber, dass das Beginn-Tag (`B-XXX`) erst dann verwendet wird, wenn es nötig ist, d.h. wenn zwei NEs vom gleichen Type adjazent sind! <span style="color:red">(Frage am Rande: Macht das die Sache schwieriger oder einfacher?)</span>

Der CoNLL2003 Shared Task unterscheidet die vier NE-Typen 
- ORG(ANIZATION) 
- PER(SON)
- LOC(ATION)
- MISC(ELLANEOUS)

### CoNLL-2003 Shared Task: Statistik der Trainingsdaten

Die Einteilung in Trainings-, Entwicklungs- und Testdaten ist beim CoNLL-2003 Shared Task vorgegeben. Um die Annotation etwas besser kennenzulernen, werfen wir einen Blick auf die Trainingsdaten.

In [8]:
print([token for token in conll2003.iob_words('eng.train',column='ne')[:50]])

[('EU', 'NNP', 'I-ORG'), ('rejects', 'VBZ', 'O'), ('German', 'JJ', 'I-MISC'), ('call', 'NN', 'O'), ('to', 'TO', 'O'), ('boycott', 'VB', 'O'), ('British', 'JJ', 'I-MISC'), ('lamb', 'NN', 'O'), ('.', '.', 'O'), ('Peter', 'NNP', 'I-PER'), ('Blackburn', 'NNP', 'I-PER'), ('BRUSSELS', 'NNP', 'I-LOC'), ('1996-08-22', 'CD', 'O'), ('The', 'DT', 'O'), ('European', 'NNP', 'I-ORG'), ('Commission', 'NNP', 'I-ORG'), ('said', 'VBD', 'O'), ('on', 'IN', 'O'), ('Thursday', 'NNP', 'O'), ('it', 'PRP', 'O'), ('disagreed', 'VBD', 'O'), ('with', 'IN', 'O'), ('German', 'JJ', 'I-MISC'), ('advice', 'NN', 'O'), ('to', 'TO', 'O'), ('consumers', 'NNS', 'O'), ('to', 'TO', 'O'), ('shun', 'VB', 'O'), ('British', 'JJ', 'I-MISC'), ('lamb', 'NN', 'O'), ('until', 'IN', 'O'), ('scientists', 'NNS', 'O'), ('determine', 'VBP', 'O'), ('whether', 'IN', 'O'), ('mad', 'JJ', 'O'), ('cow', 'NN', 'O'), ('disease', 'NN', 'O'), ('can', 'MD', 'O'), ('be', 'VB', 'O'), ('transmitted', 'VBN', 'O'), ('to', 'TO', 'O'), ('sheep', 'NN', 'O')

Es überrascht zunächst nicht, dass `O` das häufigste NE-Tag ist:

In [9]:
from nltk.probability import FreqDist

corpstat = FreqDist(conll2003.iob_words('eng.train',column='ne'))
netagstat= FreqDist([netag for word,pos,netag in conll2003.iob_words('eng.train',column='ne')])

print("Häufigste Token: ",corpstat.most_common(10),"\n")
print("Häufigkeit der NE-Tags: ",netagstat.most_common(10))

Häufigste Token:  [(('.', '.', 'O'), 7362), ((',', ',', 'O'), 7275), (('the', 'DT', 'O'), 7221), (('of', 'IN', 'O'), 3617), (('to', 'TO', 'O'), 3382), (('in', 'IN', 'O'), 3370), (('a', 'DT', 'O'), 2993), (('(', '(', 'O'), 2846), ((')', ')', 'O'), 2846), (('and', 'CC', 'O'), 2789)] 

Häufigkeit der NE-Tags:  [('O', 169578), ('I-PER', 11128), ('I-ORG', 10001), ('I-LOC', 8286), ('I-MISC', 4556), ('B-MISC', 37), ('B-ORG', 24), ('B-LOC', 11)]


Wie beim POS-Tagging ist die Ambiguität der Worttoken weitaus ausgeprägter als die Ambiguität der Wortformen:

In [10]:
# Create a dictionary to store the list of NE tags per word form
netagdict = {}

for word,pos,netag in corpstat.keys() :
    if word in netagdict :
        if netag not in netagdict[word] :
            netagdict[word] = netagdict[word] + [netag]
    else :
        netagdict[word] = [netag]
    
netagdict

{'EU': ['I-ORG'],
 'rejects': ['O'],
 'German': ['I-MISC', 'I-ORG'],
 'call': ['O'],
 'to': ['O'],
 'boycott': ['O'],
 'British': ['I-MISC', 'I-ORG', 'I-LOC'],
 'lamb': ['O'],
 '.': ['O', 'I-MISC', 'I-LOC', 'I-ORG'],
 'Peter': ['I-PER'],
 'Blackburn': ['I-PER', 'I-ORG'],
 'BRUSSELS': ['I-LOC', 'I-MISC'],
 '1996-08-22': ['O'],
 'The': ['O', 'I-LOC', 'I-ORG', 'I-MISC'],
 'European': ['I-ORG', 'I-MISC'],
 'Commission': ['I-ORG', 'I-MISC'],
 'said': ['O'],
 'on': ['O', 'I-MISC'],
 'Thursday': ['O'],
 'it': ['O'],
 'disagreed': ['O'],
 'with': ['O'],
 'advice': ['O'],
 'consumers': ['O'],
 'shun': ['O'],
 'until': ['O'],
 'scientists': ['O'],
 'determine': ['O'],
 'whether': ['O'],
 'mad': ['O'],
 'cow': ['O'],
 'disease': ['O'],
 'can': ['O'],
 'be': ['O'],
 'transmitted': ['O'],
 'sheep': ['O'],
 'Germany': ['I-LOC', 'I-ORG'],
 "'s": ['O', 'I-ORG', 'I-MISC', 'I-LOC'],
 'representative': ['O'],
 'the': ['O', 'I-ORG', 'I-PER', 'I-MISC', 'I-LOC'],
 'Union': ['I-ORG', 'I-LOC', 'O'],
 'veterin

In [11]:
# Average number of NE tags per word form
from statistics import mean 
mean([len(netaglist) for netaglist in netagdict.values()])

1.0486390382254582

In [12]:
# Average number of NE tags per word token
mean([len(netagdict[word]) for word,pos,netag in conll2003.iob_words('eng.train',column='ne')])

1.7827778077899628

In [13]:
netagdict['New']

['I-ORG', 'I-LOC', 'O', 'I-MISC']

## NER als überwacht gelerntes N-Gram-Tagging

Die einfachste Methode der NER mit überwachtem Lernen ist das N-Gram-Tagging, das wir bereits beim generischen Chunking angewandt haben. <span style="color:red">(Frage am Rande: Dafür müssen wir allerdings die in der Pipeline zuvor generierten Chunks wegschmeißen. Oder nicht?)</span> 

Der Trick hierbei besteht wieder darin, dem N-Gram-Tagger die richtigen Informationen zur Verfügung zu stellen. Das war beim Chunking das POS-Tag des Zieltokens und die Chunk-Tags der $n$ vorhergehenden Token. Bei der NER ist dagegen die **Wortform des Zieltokens** und die **NE-Tags der $n$ vorhergehenden Token** relevant. <span style="color:red">(Frage am Rande: In welches Problem wird man solch einem N-Gram-Ansatz wohl unweigerlich laufen?)</span>  

![](https://www.nltk.org/images/tag-context.png)

D.h. wir müssen die Trainings- und Testdaten erst entsprechend anpassen, bevor wir die Klassen wie beim Chunking benutzen können:

In [14]:
# Replacing the pos tag for the word form in the training data

Conll2003TaggedSentsTrain = [[(word,word,ne) for word,pos,ne in sent] for sent in conll2003.iob_sents('eng.train',column='ne') if sent]
Conll2003TaggedWordsTrain = [(word,word,ne) for sent in conll2003.iob_sents('eng.train',column='ne')for word,pos,ne in sent]
Conll2003WordsTrain = [(word) for sent in conll2003.iob_sents('eng.train',column='ne')for word,pos,ne in sent]

Conll2003IOBSentsTrain = [[(word,word,ne) for word,pos,ne in sent] for sent in conll2003.iob_sents('eng.train',column='ne') if sent]
Conll2003IOBWordsTrain = [(word,word,ne) for sent in conll2003.iob_sents('eng.train',column='ne')for word,pos,ne in sent]
Conll2003ChunkedSentsTrain = [nltk.chunk.conlltags2tree(sent) for sent in Conll2003IOBSentsTrain]

# Replacing the pos tag for the word form in the development data

Conll2003TaggedSentsDev = [[(word,word) for word,pos,ne in sent] for sent in conll2003.iob_sents('eng.testa',column='ne')]
Conll2003TaggedWordsDev = [(word,word) for sent in conll2003.iob_sents('eng.testa',column='ne')for word,pos,ne in sent]
Conll2003WordsDev = [(word) for sent in conll2003.iob_sents('eng.testa',column='ne')for word,pos,ne in sent]

Conll2003IOBSentsDev = [[(word,word,ne) for word,pos,ne in sent] for sent in conll2003.iob_sents('eng.testa',column='ne') if sent]
Conll2003IOBWordsDev = [(word,word,ne) for sent in conll2003.iob_sents('eng.testa',column='ne')for word,pos,ne in sent]
Conll2003ChunkedSentsDev = [nltk.chunk.conlltags2tree(sent) for sent in Conll2003IOBSentsDev]

# Replacing the pos tag for the word form in the test data

Conll2003TaggedSentsTest = [[(word,word) for word,pos,ne in sent] for sent in conll2003.iob_sents('eng.testb',column='ne')]
Conll2003TaggedWordsTest = [(word,word) for sent in conll2003.iob_sents('eng.testb',column='ne')for word,pos,ne in sent]
Conll2003WordsTest = [(word) for sent in conll2003.iob_sents('eng.testb',column='ne')for word,pos,ne in sent]

Conll2003IOBSentsTest = [[(word,word,ne) for word,pos,ne in sent] for sent in conll2003.iob_sents('eng.testb',column='ne') if sent]
Conll2003IOBWordsTest = [(word,word,ne) for sent in conll2003.iob_sents('eng.testb',column='ne')for word,pos,ne in sent]
Conll2003ChunkedSentsTest = [nltk.chunk.conlltags2tree(sent) for sent in Conll2003IOBSentsTest]

In [15]:
Conll2003TaggedSentsTrain

[[('EU', 'EU', 'I-ORG'),
  ('rejects', 'rejects', 'O'),
  ('German', 'German', 'I-MISC'),
  ('call', 'call', 'O'),
  ('to', 'to', 'O'),
  ('boycott', 'boycott', 'O'),
  ('British', 'British', 'I-MISC'),
  ('lamb', 'lamb', 'O'),
  ('.', '.', 'O')],
 [('Peter', 'Peter', 'I-PER'), ('Blackburn', 'Blackburn', 'I-PER')],
 [('BRUSSELS', 'BRUSSELS', 'I-LOC'), ('1996-08-22', '1996-08-22', 'O')],
 [('The', 'The', 'O'),
  ('European', 'European', 'I-ORG'),
  ('Commission', 'Commission', 'I-ORG'),
  ('said', 'said', 'O'),
  ('on', 'on', 'O'),
  ('Thursday', 'Thursday', 'O'),
  ('it', 'it', 'O'),
  ('disagreed', 'disagreed', 'O'),
  ('with', 'with', 'O'),
  ('German', 'German', 'I-MISC'),
  ('advice', 'advice', 'O'),
  ('to', 'to', 'O'),
  ('consumers', 'consumers', 'O'),
  ('to', 'to', 'O'),
  ('shun', 'shun', 'O'),
  ('British', 'British', 'I-MISC'),
  ('lamb', 'lamb', 'O'),
  ('until', 'until', 'O'),
  ('scientists', 'scientists', 'O'),
  ('determine', 'determine', 'O'),
  ('whether', 'whether',

Jetzt können wir die Chunker-Klassen (und nützliche Methoden wie `evaluate()`) direkt verwenden. Als erstes implementieren wir den `DefaultChunker` mit dem häufigsten NE-Tag, nämlich `O`:

In [17]:
# Adapted from https://www.nltk.org/book/ch07.html
class DefaultChunker(nltk.ChunkParserI):
    def __init__(self, default): 
        self.tagger = nltk.DefaultTagger(default)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

default_ne_chunker = DefaultChunker('O')
print(default_ne_chunker.evaluate(Conll2003ChunkedSentsDev))

ChunkParse score:
    IOB Accuracy:  83.3%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


Wie nicht anders zu erwarten, erzielt der `DefaulChunker` eine recht ansehnliche IOB-Accurracy. Doch da Precision, Recall und F-Measure in Bezug auf tatsächliche Chunks (also alles außer `O`) ermittelt wird, ist deren Wert hier sehr viel geringer, nämlich jeweils "0.0%%".

Glücklicherweise benötigen wir den `DefaultChunker` nur als letztes Element der Backoff-Kette für die N-Gram-Tagger/Chunker:

In [18]:
# Taken from https://www.nltk.org/book/ch07.html
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data,
                                         backoff=nltk.DefaultTagger('O'))

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
    
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data,
                                        backoff=nltk.UnigramTagger(train_data,
                                                                   backoff=nltk.DefaultTagger('O')))

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

class TrigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.TrigramTagger(train_data,
                                         backoff=nltk.BigramTagger(train_data,
                                                                   backoff=nltk.UnigramTagger(train_data,
                                                                                              backoff=nltk.DefaultTagger('O'))))
        
    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

unigram_ne_chunker = UnigramChunker(Conll2003ChunkedSentsTrain)
bigram_ne_chunker = BigramChunker(Conll2003ChunkedSentsTrain)
trigram_ne_chunker = TrigramChunker(Conll2003ChunkedSentsTrain)
print("Unigram:",unigram_ne_chunker.evaluate(Conll2003ChunkedSentsDev))
print("Bigram :",bigram_ne_chunker.evaluate(Conll2003ChunkedSentsDev))
print("Trigram :",trigram_ne_chunker.evaluate(Conll2003ChunkedSentsDev))

Unigram: ChunkParse score:
    IOB Accuracy:  93.5%%
    Precision:     67.2%%
    Recall:        63.6%%
    F-Measure:     65.4%%
Bigram : ChunkParse score:
    IOB Accuracy:  94.1%%
    Precision:     74.4%%
    Recall:        67.1%%
    F-Measure:     70.6%%
Trigram : ChunkParse score:
    IOB Accuracy:  94.2%%
    Precision:     75.1%%
    Recall:        67.5%%
    F-Measure:     71.1%%


Diese Werte sehen schon sehr viel besser aus, liegen sogar oberhalb der **Baseline für die Testdaten** (Precision: 71.91%, Recall: 50.90%, F1-Masure: 59.61%). Werfen wir einen Blick auf die Verwechslungsmatrix, um Verbesserungsmöglichkeiten zu finden:

In [19]:
from nltk.metrics import ConfusionMatrix
from nltk import conlltags2tree, tree2conlltags

#gold = [netag for sent in Conll2003IOBSentsTest for word,word,netag in sent] # Missing B-XXX tags
cleanedGoldSents = [tree2conlltags(conlltags2tree(sent)) for sent in Conll2003IOBSentsDev]
gold = [netag for sent in cleanedGoldSents for word,word,netag in sent]
test = [netag for sent in Conll2003TaggedSentsDev for word,word,netag in tree2conlltags(bigram_ne_chunker.parse(sent))]

print(ConfusionMatrix(gold, test).pretty_format(show_percents=True, values_in_chart=True, truncate=15, sort_by_count=True))

       |                                         B             I        |
       |             B      B      B      I      -      I      -      I |
       |             -      -      -      -      M      -      M      - |
       |             P      L      O      P      I      O      I      L |
       |             E      O      R      E      S      R      S      O |
       |      O      R      C      G      R      C      G      C      C |
-------+----------------------------------------------------------------+
     O | <83.1%>  0.0%   0.0%   0.1%      .   0.0%   0.1%   0.0%   0.0% |
 B-PER |   0.9%  <2.6%>  0.0%   0.0%      .   0.0%      .      .      . |
 B-LOC |   0.4%   0.0%  <2.9%>  0.2%      .   0.0%   0.0%      .      . |
 B-ORG |   0.5%   0.0%   0.2%  <1.9%>     .   0.0%      .      .      . |
 I-PER |   1.1%   0.1%   0.0%   0.0%  <1.3%>  0.0%   0.0%      .      . |
B-MISC |   0.3%   0.0%   0.1%   0.0%      .  <1.3%>  0.0%   0.0%      . |
 I-ORG |   0.4%   0.0%   0.1%   0.3%  

Aus der Verwechlungsmatrix geht u.a. hervor, dass viele der Personen-Token fälschlicherweise mit `O` getagged werden (siehe Zeile B-PER und I-PER). Offensichtlich enthalten die Trainingsdaten nicht die entsprechenden Instanzen. Wie können wir diese Lücken füllen?

### NER mit orthographischem Wissen

Ein Weg, entgangene NEs aufzuspüren, besteht darin, deren orthographische Eigenschaften zu berücksichtigen. Das ist im Englischen relativ einfach, denn Eigennamen werden in der Regel groß geschrieben (auch als Adjektiv) und alles andere klein.

Man ist versucht, dieses Wissen mit dem `RexpParser` umzusetzen und Chunks mit großgeschriebenen Wortformen dem NE-Tag `PER` zuzuordnen (hier können wir uns das IOB-Schema sparen):

In [20]:
orthgram = """
PER: {<[A-Z].*>+}
"""

regexp_ne_chunker = nltk.RegexpParser(orthgram,loop=1)

test = [('EU', 'EU'),
  ('rejects', 'rejects'),
  ('German', 'German'),
  ('call', 'call'),
  ('to', 'to'),
  ('boycott', 'boycott'),
  ('British', 'British'),
  ('lamb', 'lamb'),
  ('.', '.')]

print(regexp_ne_chunker.parse(test))

(S
  (PER EU/EU)
  rejects/rejects
  (PER German/German)
  call/call
  to/to
  boycott/boycott
  (PER British/British)
  lamb/lamb
  ./.)


Leider sehe ich keine einfache Möglichkeit, die Ergebnisse des `RegexpParser` in die oben dargestellte Backoff-Sequenz zu integrieren.

Stattdessen empfiehlt es sich, auf `RegexpTagger` zurückzugreifen.

### <span style="color:red">Aufgaben I: Regexp-Tagger</span>

<span style="color:red">A1:</span> Vervollständigen Sie die folgende Initialisierung eines Regexp-Taggers so, dass großgeschriebene Wortformen mit einem möglichst sinnvollen NE-Tag versehen werden!

In [57]:
from nltk import RegexpTagger
rnet = RegexpTagger([
    # Lösung A1
    #(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
    ('[A-Z].*?', 'I-PER')               # starting with capitals
],backoff=nltk.DefaultTagger('O'))

In [58]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data,
                                         backoff=rnet)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
    
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data,
                                        backoff=nltk.UnigramTagger(train_data,
                                                                   backoff=rnet))

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

class TrigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.TrigramTagger(train_data,
                                         backoff=nltk.BigramTagger(train_data,
                                                                   backoff=nltk.UnigramTagger(train_data,
                                                                                              backoff=rnet)))

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
    
unigram_ne_chunker = UnigramChunker(Conll2003ChunkedSentsTrain)
bigram_ne_chunker = BigramChunker(Conll2003ChunkedSentsTrain)
trigram_ne_chunker = TrigramChunker(Conll2003ChunkedSentsTrain)
print("Unigram:",unigram_ne_chunker.evaluate(Conll2003ChunkedSentsDev))
print("Bigram :",bigram_ne_chunker.evaluate(Conll2003ChunkedSentsDev))
print("Trigram :",trigram_ne_chunker.evaluate(Conll2003ChunkedSentsDev))

Unigram: ChunkParse score:
    IOB Accuracy:  95.0%%
    Precision:     65.2%%
    Recall:        76.0%%
    F-Measure:     70.2%%
Bigram : ChunkParse score:
    IOB Accuracy:  95.6%%
    Precision:     71.0%%
    Recall:        79.6%%
    F-Measure:     75.0%%
Trigram : ChunkParse score:
    IOB Accuracy:  95.7%%
    Precision:     71.5%%
    Recall:        79.9%%
    F-Measure:     75.5%%


Wenn Sie die Tagging-Regeln richtig aufgeschrieben haben sollten, Sie eine Verbesserung gegenüber dem ursprünglichen System feststellen, die sich auch in der Verwechslungsmatrix widerspiegelt.

In [59]:
from nltk.metrics import ConfusionMatrix
from nltk import conlltags2tree, tree2conlltags

#gold = [netag for sent in Conll2003IOBSentsTest for word,word,netag in sent] # Missing B-XXX tags
cleanedGoldSents = [tree2conlltags(conlltags2tree(sent)) for sent in Conll2003IOBSentsDev]
gold = [netag for sent in cleanedGoldSents for word,word,netag in sent]
test = [netag for sent in Conll2003TaggedSentsDev for word,word,netag in tree2conlltags(trigram_ne_chunker.parse(sent))]

print(ConfusionMatrix(gold, test).pretty_format(show_percents=True, values_in_chart=True, truncate=15, sort_by_count=True))

       |                                         B             I        |
       |             B      B      B      I      -      I      -      I |
       |             -      -      -      -      M      -      M      - |
       |             P      L      O      P      I      O      I      L |
       |             E      O      R      E      S      R      S      O |
       |      O      R      C      G      R      C      G      C      C |
-------+----------------------------------------------------------------+
     O | <82.5%>  0.4%   0.0%   0.1%   0.1%   0.0%   0.1%   0.0%   0.0% |
 B-PER |   0.0%  <3.5%>  0.0%   0.0%   0.0%   0.0%      .      .      . |
 B-LOC |   0.0%   0.4%  <2.9%>  0.2%      .   0.1%   0.0%      .      . |
 B-ORG |   0.0%   0.5%   0.2%  <1.9%>  0.0%   0.1%      .      .      . |
 I-PER |   0.0%   0.1%   0.0%   0.0%  <2.4%>  0.0%   0.0%      .      . |
B-MISC |   0.1%   0.2%   0.1%   0.1%      .  <1.3%>  0.0%   0.0%      . |
 I-ORG |   0.2%   0.1%   0.1%   0.3%  

### NER mit explizitem Typen-Wissen

Die orthographische Schreibung ist sicherlich in manchen Sprachen ein gutes Indiz für das Vorliegen einer NE, aber der NE-Typ lässt sich damit wohl kaum zuverlässig ermitteln. Dafür benötigt man früher oder später (abhängig von der Größe und Abdeckung der Lerndaten) explizites Typen-Wissen, d.h. Listen von NEs, die einem NE-Typ zugeordnet sind.

NLTK stellt zwei `corpus`-Module mit solchem Typen-Wissen zur Verfügung:

- Geographische Bezeichnungen (zusammen ein sogenanntes [Gazetteer](https://en.wikipedia.org/wiki/Gazetteer)): 

In [60]:
from nltk.corpus import gazetteers
gazetteers.words()

['Alberta',
 'British Columbia',
 'Manitoba',
 'New Brunswick',
 'Newfoundland and Labrador',
 'Nova Scotia',
 'Northwest Territories',
 'Nunavut',
 'Ontario',
 'Prince Edward Island',
 'Quebec',
 'Saskatchewan',
 'Yukon',
 'Abkhazia',
 'Afghanistan',
 'Akrotiri',
 'Akrotiri and Dhekelia',
 'Aland',
 'Aland Islands',
 'Albania',
 'Algeria',
 'America',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Ascension Island',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Barbuda',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Caicos Islands',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas 

- Vornamen: 

In [61]:
from nltk.corpus import names 
names.words()

['Abagael',
 'Abagail',
 'Abbe',
 'Abbey',
 'Abbi',
 'Abbie',
 'Abby',
 'Abigael',
 'Abigail',
 'Abigale',
 'Abra',
 'Acacia',
 'Ada',
 'Adah',
 'Adaline',
 'Adara',
 'Addie',
 'Addis',
 'Adel',
 'Adela',
 'Adelaide',
 'Adele',
 'Adelice',
 'Adelina',
 'Adelind',
 'Adeline',
 'Adella',
 'Adelle',
 'Adena',
 'Adey',
 'Adi',
 'Adiana',
 'Adina',
 'Adora',
 'Adore',
 'Adoree',
 'Adorne',
 'Adrea',
 'Adria',
 'Adriaens',
 'Adrian',
 'Adriana',
 'Adriane',
 'Adrianna',
 'Adrianne',
 'Adrien',
 'Adriena',
 'Adrienne',
 'Aeriel',
 'Aeriela',
 'Aeriell',
 'Ag',
 'Agace',
 'Agata',
 'Agatha',
 'Agathe',
 'Aggi',
 'Aggie',
 'Aggy',
 'Agna',
 'Agnella',
 'Agnes',
 'Agnese',
 'Agnesse',
 'Agneta',
 'Agnola',
 'Agretha',
 'Aida',
 'Aidan',
 'Aigneis',
 'Aila',
 'Aile',
 'Ailee',
 'Aileen',
 'Ailene',
 'Ailey',
 'Aili',
 'Ailina',
 'Ailyn',
 'Aime',
 'Aimee',
 'Aimil',
 'Aina',
 'Aindrea',
 'Ainslee',
 'Ainsley',
 'Ainslie',
 'Ajay',
 'Alaine',
 'Alameda',
 'Alana',
 'Alanah',
 'Alane',
 'Alanna',
 

Leider enthält das NLTK keine NE-Liste für den Typ ORGANIZATION. Hier wie auch bei den anderen NE-Typen könnte man z.B. auf die Wikipedia (oder besser die [DBPedia](https://wiki.dbpedia.org/)) zurückgreifen. 

### <span style="color:red">Aufgaben II: Explizites Typen-Wissen</span>

<span style="color:red">A2:</span> Integrieren Sie das Typen-Wissen in `gazetteers` und `names` (soweit wie möglich) in die oben verwendete Backoff-Sequenz!

In [64]:
# Lösung A2
n = names.words()             # nltk names as a list
g = gazetteers.words()        # nltk gazeteers as a list

from nltk import RegexpTagger
rnet = RegexpTagger([
    # Lösung A1
    #(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),          # cardinal numbers
    (r"(?=("+'|'.join(n)+r"))", 'I-PER'),       # corpus names seperated by |
    (r"(?=("+'|'.join(g)+r"))", 'I-ORG'),       # corpus gazeteers seperated by |
    ('[A-Z].*?', 'I-PER')                      # starting with capitals
],backoff=nltk.DefaultTagger('O'))


In [66]:
#class UnigramChunker(nltk.ChunkParserI):
#    def __init__(self, train_sents): 
#        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
#                      for sent in train_sents]
#        self.tagger = nltk.UnigramTagger(train_data,
#                                         backoff=rnet)#

#    def parse(self, sentence):
#        pos_tags = [pos for (word,pos) in sentence]
#        tagged_pos_tags = self.tagger.tag(pos_tags)
#        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
#        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
#                     in zip(sentence, chunktags)]
#        return nltk.chunk.conlltags2tree(conlltags)

#unigram_ne_chunker = UnigramChunker(Conll2003ChunkedSentsTrain)

### Abschließende Evaluation mit den Testdaten

Nach Abschluss der Entwicklung des NER-Systems können wir die Evaluation anhand der Testdaten durchführen: 

In [67]:
print("Unigram:",unigram_ne_chunker.evaluate(Conll2003ChunkedSentsTest))
print("Bigram :",bigram_ne_chunker.evaluate(Conll2003ChunkedSentsTest))
print("Trigram :",trigram_ne_chunker.evaluate(Conll2003ChunkedSentsTest))

Unigram: ChunkParse score:
    IOB Accuracy:  93.2%%
    Precision:     56.1%%
    Recall:        69.3%%
    F-Measure:     62.0%%
Bigram : ChunkParse score:
    IOB Accuracy:  93.7%%
    Precision:     61.2%%
    Recall:        71.7%%
    F-Measure:     66.1%%
Trigram : ChunkParse score:
    IOB Accuracy:  93.7%%
    Precision:     61.5%%
    Recall:        71.9%%
    F-Measure:     66.3%%


Die Baseline beim CoNLL-2003 Shared Tast wird aus den NEs der Lerndaten mit eindeutigem NE-Typ gebildet ("entities with unique class in the training data"):

- Precision: 71.91% 
- Recall: 50.90% 
- F1-Masure: 59.61%

Wenn alles gut gegangen ist, sollte das finale NER-System deutlich über der Baseline liegen und sogar vor dem letzten Platz im Wettbewerb landen. 